In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "3" # 0 -> 1, 3 -> 2


In [2]:
import torch
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)
from trl import DPOTrainer, DPOConfig
import json
from datasets import Dataset
from sklearn.model_selection import train_test_split
from accelerate import Accelerator

2024-05-24 13:16:22.946350: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-24 13:16:23.522790: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[2024-05-24 13:16:24,261] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/home/rfaro/anaconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible


In [3]:
torch.cuda.empty_cache()


In [4]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", torch_dtype = "auto", trust_remote_code = True)
tokenizer.add_special_tokens({"pad_token": "<pad>"})

/home/rfaro/anaconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


1

In [5]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
)

policy_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    device_map={"": Accelerator().local_process_index}
)

policy_model.config.use_cache = False


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/home/rfaro/anaconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
#policy_model = prepare_model_for_kbit_training(policy_model)

policy_model.config.pad_token_id = tokenizer.pad_token_id
policy_model.resize_token_embeddings(len(tokenizer))

#policy_model.load_state_dict(torch.load("/home/rfaro/anaconda3/EPFL_ML/ml-project-2-betterthanpoli/model_state_2_large_v2.pt", map_location="cuda:0"))

Embedding(50296, 2560)

In [7]:
policy_model

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(50296, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiSdpaAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_la

In [8]:
num_params_requires_grad = sum(p.numel() for p in policy_model.parameters() if p.requires_grad)

print("Number of parameters requiring gradients:", num_params_requires_grad)


Number of parameters requiring gradients: 257734776


In [9]:
peft_config = LoraConfig(
        r=16,
        lora_alpha=8,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["q_proj", "k_proj", "v_proj", "dense", "fc1", "fc2"]
)

In [ ]:
# bnb_config_ref = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype=torch.bfloat16,
#         bnb_4bit_use_double_quant=True
# )

# reference_model = AutoModelForCausalLM.from_pretrained(
#     "gpt2-large",
#     quantization_config=bnb_config,
#     device_map="auto",
#     trust_remote_code = True,
#     low_cpu_mem_usage=True,
# )

In [20]:
training_arguments = DPOConfig(
    output_dir="./results",
        evaluation_strategy="steps",
        do_eval=True,
        #optim="adamw_bnb_8bit",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        per_device_eval_batch_size=1,
        log_level="debug",
        save_steps=10,
        logging_steps=10,
        learning_rate=1e-5,
        eval_steps=20,
        max_steps = 10,
        num_train_epochs=3,
        warmup_steps=20,
        lr_scheduler_type="cosine",
        remove_unused_columns=False,
        max_length=2048,
        max_prompt_length=2048
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [23]:
dict_dataset = json.load(open("/home/rfaro/anaconda3/Marco/dpo_dataset.json", "r"))
print(len(dict_dataset['prompt']))

21596


In [24]:
# truncate sentences in dataset that are longer than 1024 tokens
filtered_dataset = {
    "prompt": [],
    "chosen": [],
    "rejected": []

}

for i in range(len(dict_dataset["prompt"])):
    if len(tokenizer(dict_dataset["prompt"][i])["input_ids"]) > 512:
        continue
    if len(tokenizer(dict_dataset["chosen"][i])["input_ids"]) > 512:
        continue
    if len(tokenizer(dict_dataset["rejected"][i])["input_ids"]) > 512:
        continue
    filtered_dataset["prompt"].append(dict_dataset["prompt"][i])
    filtered_dataset["chosen"].append(dict_dataset["chosen"][i])
    filtered_dataset["rejected"].append(dict_dataset["rejected"][i])
        

In [13]:
print(len(filtered_dataset['prompt']))

17763


In [14]:
21596*0.002

43.192

In [25]:
dataset = Dataset.from_dict(filtered_dataset)
train_set, val_set = train_test_split(dataset, test_size=0.002)
train_set = Dataset.from_dict(train_set)
val_set = Dataset.from_dict(val_set)

In [15]:
dataset[0]

{'prompt': 'Question: Consider the following contains function defined on Iterable (in particular, it accepts both Vector and List).  def contains[A](l: Iterable[A], elem: A): Boolean =   val n = l.size   if n <= 5 then     for i <- l do       if i == elem then         return true     false   else     val (p0, p1) = parallel(       contains(l.take(n / 2), elem),       contains(l.drop(n / 2), elem)     )   p0 || p1 Let $n$$n$ be the size of l. Assume that drop and take run in $\\Theta(1)$ on Vector and $\\Theta(n)$ on List. What is the asymptotic depth of contains if it is called on a List?',
 'chosen': 'When `contains` is called on a List, the `drop` and `take` operations run in $\\Theta(n)$ time. \n\nIn the case where $n \\leq 5$, the loop runs in $\\Theta(n)$ time since it iterates over the elements of the list once.\n\nIn the case where $n > 5$, the list is split into two parts of roughly equal size, and the `contains` function is recursively called on each half. This splitting proc

In [26]:
# print number of trainable parameters of policy model
num_params_requires_grad = sum(p.numel() for p in policy_model.parameters() if p.requires_grad)
print("Number of parameters requiring gradients:", num_params_requires_grad)

Number of parameters requiring gradients: 23592960


In [16]:
policy_model

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(50296, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiSdpaAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_la

In [27]:
trainer = DPOTrainer(
    policy_model,
    ref_model=None,
    max_length = 512,
    #max_length=8196,
    #max_prompt_length=4096,
    args=training_arguments,
    beta=0.1,
    peft_config=peft_config,
    train_dataset=train_set,
    eval_dataset=val_set,
    tokenizer=tokenizer,
    #force_use_ref_model=True
)


/home/rfaro/anaconda3/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in DPOTrainer, please use the DPOConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/rfaro/anaconda3/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:357: UserWarning: You passed `max_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(


Map:   0%|          | 0/17727 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs


In [23]:
tokenizer.pad_token

'<pad>'

In [28]:
trainer.train()

Currently training with a batch size of: 1
***** Running training *****
  Num examples = 17,727
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 10
  Number of trainable parameters = 23,592,960
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: robinfaro (deepsensing). Use `wandb login --relogin` to force relogin


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss


Saving model checkpoint to ./results/checkpoint-10
/home/rfaro/anaconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /home/rfaro/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/ef382358ec9e382308935a992d908de099b64c23/config.json
Model config PhiConfig {
  "_name_or_path": "microsoft/phi-2",
  "architectures": [
    "PhiForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "hidden_act": "gelu_new",
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 10240,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "phi",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_h

TrainOutput(global_step=10, training_loss=0.6930822372436524, metrics={'train_runtime': 55.9616, 'train_samples_per_second': 0.715, 'train_steps_per_second': 0.179, 'total_flos': 0.0, 'train_loss': 0.6930822372436524, 'epoch': 0.002256444970948271})